In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [2]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_10Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [3]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [4]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [6]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [7]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [8]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    
    global df
    for i in range(10):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-dca8b71f45e14bb9ac5c9a73681a1360","object":"text_completion","created":453568,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.851506 seconds 


Prompt==> Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-9aac14c94ce24761bce12a763c2f75a5","object":"text_completion","created":453568,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/sna

Prompt==> Problem: Write a function in java to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-996509ccf985413891ef570c395525c6","object":"text_completion","created":453580,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\npublic class Main {\n    public static void main(String[] args) {\n        String s = \"Hello World\";\n        System.out.println(reverseString(s));\n    }\n\n    public static String reverseString(String s) {\n        String reversed = \"\";\n        for (int i = s.length() - 1; i >= 0; i--) {\n            reversed += s.charAt(i);\n        }\n        return reversed;\n    }\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":146,"completion_tokens":115}}
Execution time: 5.953223 seconds 


Prompt==> Problem: Write a function in java to calculate the factorial of a given non-negative integ

Prompt==> Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-f9a0ff0df053494fbaa4491d9de6f8f1","object":"text_completion","created":453593,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```java\npublic class BinarySearch {\n    public static int binarySearch(int[] arr, int target) {\n        int left = 0;\n        int right = arr.length - 1;\n\n        while (left <= right) {\n            int mid = left + (right - left) / 2;\n\n            if (arr[mid] == target) {\n                return mid;\n            } else if (arr[mid] < target) {\n                left = mid + 1;\n            } else {\n                right = mid - 1;\n            }\n        }\n\n        return -1;\n    }\n\n    public static void main(String[] args) {\n        int[

Prompt==> Problem: Write a function in javascript to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-a6a9fdcb8ef14049b2a2f60bb396c806","object":"text_completion","created":453617,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction reverseString(s) {\n    return s.split(\"\").reverse().join(\"\");\n}\n\nconsole.log(reverseString(\"Hello World\")); // Output: \"dlroW olleH\"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.671413 seconds 


Prompt==> Problem: Write a function in javascript to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-51b4bbc592b54316b1ef6ef42b17efb0","object":"text_completion","created":453617,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.

Prompt==> Problem: Write a function in Ruby to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-c52aa069130d4b0593818740cbeff3f3","object":"text_completion","created":453632,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n)\n  if n == 0\n    1\n  else\n    n * factorial(n-1)\n  end\nend\n\nputs factorial(5)","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":41,"total_tokens":88,"completion_tokens":47}}
Execution time: 2.418298 seconds 


Prompt==> Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-f91cb74e443d411eb178efcf77ef3717","object":"text_completion","created":453624,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c

...

Prompt==> Problem: Implement an autocomplete feature in react for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions). 
 {"id":"cmpl-6b9a0c1355cf4dbd92db2e927afeb997","object":"text_completion","created":454441,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can implement an autocomplete feature in React using a controlled component and the useState hook.\n\n```jsx\nimport React, { useState } from 'react';\n\nconst Autocomplete = () => {\n  const [inputValue, setInputValue] = useState('');\n  const [suggestions, setSuggestions] = useState([]);\n\n  const handleInputChange = (event) => {\n    const userInput = event.target.value;\n\n    // Filter suggestions based on user input\n    const filteredSuggestions = suggestions.filter(\n      (suggestion) =>\n        suggestion.toLowerCase().inde

Prompt==> Problem: Write a function in react that dynamically creates an HTML element and appends it to the document.Input: String, element type (e.g., 'div', 'button').Output: None (update the HTML document). 
 {"id":"cmpl-3fe8ee18858c4c95b81914ccb3dfb082","object":"text_completion","created":454453,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create a function in React that creates an HTML element and appends it to the document:\n\n```jsx\nimport React from 'react';\n\nclass App extends React.Component {\n  createElement(type, text) {\n    const element = document.createElement(type);\n    element.textContent = text;\n    document.body.appendChild(element);\n  }\n\n  render() {\n    return (\n      <div>\n        <button onClick={() => this.createElement('div', 'Hello, World!')}>\n          Create Div\n        </button>\n        <button onClick=

Prompt==> Problem: Implement form validation in angular to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid. 
 {"id":"cmpl-111d2e20eac34a83adfc6cae64cc1fc5","object":"text_completion","created":454470,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\nIn Angular, you can use FormGroup and FormControl to create a form and validate it. Here is a simple example:\n\n```typescript\nimport { Component } from '@angular/core';\nimport { FormGroup, FormControl, Validators } from '@angular/forms';\n\n@Component({\n  selector: 'app-root',\n  templateUrl: './app.component.html',\n  styleUrls: ['./app.component.css']\n})\nexport class AppComponent {\n  // Create a form with two fields: name and email\n  form = new FormGroup({\n    name: new FormControl('', Validators.required),\n    email: new FormControl('', [

Prompt==> Problem: Write a function in jQuery to fetch data from an API and display it on a webpage.Input: API endpoint URL.Output: None (update the webpage with fetched data). 
 {"id":"cmpl-196703f4a4c049c4b7a843db7cb225d2","object":"text_completion","created":454485,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can fetch data from an API using jQuery and display it on a webpage:\n\n```javascript\n$(document).ready(function() {\n    $.ajax({\n        url: 'https://api.example.com/data', // replace with your API endpoint URL\n        type: 'GET',\n        dataType: 'json',\n        success: function(data) {\n            // Assuming the API returns an array of objects\n            // and each object has a 'name' property\n            $.each(data, function(index, item) {\n                // Append each item's name to the body of the webpage\n            

Prompt==> Problem: Create a function in jQuery to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element). 
 {"id":"cmpl-82de0cf79be24429bde00cb93f89d910","object":"text_completion","created":454501,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nExample:\nHTML:\n<button id=\"toggleButton\">Toggle Class</button>\n<div id=\"myDiv\" class=\"myClass\">Hello World</div>\n\njQuery:\n$(document).ready(function() {\n    $(\"#toggleButton\").click(function() {\n        $(\"#myDiv\").toggleClass(\"myClass\");\n    });\n});\n\nIn this example, when the button with id \"toggleButton\" is clicked, the class \"myClass\" of the div with id \"myDiv\" will be toggled. If the div has the class \"myClass\", it will be removed, and if it doesn't have the class \"myClass\", it will be added.","logprobs":null

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))